In [2]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import cv2 as cv
import scipy.io as sio
import skimage.io as skio
from skimage import exposure

%matplotlib widget

import sys  
sys.path.insert(0, 'lib')
from iplabs import IPLabViewer as viewer

# Print colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
# Load the entire sim dataset into a numpy array
def load_dataset(path_to_raw):
    ls = []
    path_to_raw = Path(path_to_raw)

    # Read images and resize them
    print('Collecting images...   0%', end='\r')
    subdir_list = [f for f in path_to_raw.glob('*')]
    for i, f in enumerate(subdir_list):
        img = cv.imread(str(f), cv.IMREAD_GRAYSCALE)
        sz = min(img.shape)//2
        img = cv.resize(img[img.shape[0]//2 - sz:img.shape[0]//2 + sz, img.shape[1]//2 - sz:img.shape[1]//2 + sz], (512, 512))
        img = img / np.max(img)
        ls.append(img)
        print(f'Collecting images... {i/len(subdir_list)*100:3.0f}%', end='\r')
    
    print(f'Collecting images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
        
    return np.array(ls)

In [4]:
dataset = load_dataset('DNN4SIM_data/DIV2K')

### Test img

In [71]:
# tif
img = skio.imread('DNN4SIM_data/sim_test_img_2.tif')
img.shape

(2, 1028, 1054)

In [76]:
viewer(img[0])

Button(description='Show Widgets', style=ButtonStyle())

In [77]:
img = img[0]
img.shape

(1028, 1054)

In [78]:
img = img[:1024, :1024]
img.shape

(1024, 1024)

In [68]:
# png
img = cv.imread('DNN4SIM_data/sim_test_img_2.png', cv.IMREAD_GRAYSCALE)
#img=img[img.shape[0]-img.shape[1]:,:]
img.shape

(1024, 1024)

In [15]:
img = cv.resize(img, dsize=(1024,1024), interpolation=cv.INTER_LANCZOS4)

In [79]:
img = img/np.max(img)
print(img.min(), img.max())

0.0 1.0


In [17]:
# invert
img = img.max() - img

In [82]:
img_up = exposure.equalize_adapthist(img, clip_limit=0.05)

In [60]:
img[img > 0.5] = 1
img[img <= 0.5] += 0.5

In [83]:
plt.close('all')
view = viewer(img_up, cmap='viridis')

Button(description='Show Widgets', style=ButtonStyle())

In [84]:
cv.imwrite('DNN4SIM_data/sim_test_img_2.png', img_up*255)

True

# Save dataset as .mat file

In [5]:
for i in range(4):
    with open(f'DNN4SIM_data/DIV2K_512_{i}.mat', 'wb') as f:
        sio.savemat(f, {'data' : dataset[i*dataset.shape[0]//4:(i+1)*dataset.shape[0]//4]})

# Generating the rosette

In [19]:
rosette_512 = sio.loadmat(f'DNN4SIM_data/rosette_512.mat')['rosette']
rosette_1024 = sio.loadmat(f'DNN4SIM_data/rosette_1024.mat')['rosette']
rosette_512 = cv.morphologyEx(rosette_512, cv.MORPH_CLOSE, np.ones((3,3)))
rosette_1024 = cv.morphologyEx(rosette_1024, cv.MORPH_CLOSE, np.ones((7,7)))

In [24]:
plt.close('all')
view = viewer([rosette_512, rosette_1024], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

In [25]:
with open('DNN4SIM_data/rosette_512.npy', 'wb') as f:
    np.save(f, rosette_512)
with open('DNN4SIM_data/rosette_1024.npy', 'wb') as f:
    np.save(f, rosette_1024)

# Histogram equalization

In [3]:
feats = np.load('DNN4SIM_data/features.npy')

In [29]:
for i in range(feats.shape[0]):
    feats[i] = exposure.equalize_adapthist(feats[i], clip_limit=0.1)

In [ ]:
with open('DNN4SIM_data/data_reduced/features_1.npy', 'wb') as f:
    np.save(f, feats[:feats.shape[0]//2])
with open('DNN4SIM_data/data_reduced/features_2.npy', 'wb') as f:
    np.save(f, feats[feats.shape[0]//2:])

In [5]:
labs = np.load('DNN4SIM_data/labels.npy')

In [30]:
for i in range(labs.shape[0]):
    labs[i] = exposure.equalize_adapthist(labs[i], clip_limit=0.1)

In [34]:
with open('DNN4SIM_data/data_reduced/labels_1.npy', 'wb') as f:
    np.save(f, labs[:labs.shape[0]//2])
with open('DNN4SIM_data/data_reduced/labels_2.npy', 'wb') as f:
    np.save(f, labs[labs.shape[0]//2:])

In [4]:
wf = np.load('DNN4SIM_data/wf.npy')

In [8]:
for i in range(wf.shape[0]):
    wf[i] = exposure.equalize_adapthist((wf[i] - np.min(wf[i])) / (np.max(wf[i]) - np.min(wf[i])), clip_limit=0.1)

In [10]:
with open('DNN4SIM_data/data_reduced/wf_1.npy', 'wb') as f:
    np.save(f, wf[:wf.shape[0]//2])
with open('DNN4SIM_data/data_reduced/wf_2.npy', 'wb') as f:
    np.save(f, wf[wf.shape[0]//2:])

In [9]:
plt.close('all')
view = viewer([wf[253]], hist=True, cmap='viridis')

# Generate NumPy dataset from reconstruction .mat files

In [2]:
rosette_1024 = np.load('DNN4SIM_data/rosette_1024.npy')
rosette_512 = np.load('DNN4SIM_data/rosette_512.npy')
def apply_rosette(img, rosette):
    FT = np.fft.fftshift(np.fft.fft2(img))
    FT[rosette == 0] = 1e-5
    return np.real(np.fft.ifft2(np.fft.ifftshift(FT)))

def apply_fft(img):
    FT = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    FT[FT == 0] = 1e-5
    return 10*np.log10(FT)

def process_img(img):
    return (img - np.mean(img)) / np.std(img)

In [3]:
ds_recons = []
#ds = []
ds_wf = []

snr = 15

for i in range(4):
    ds_recons.append(sio.loadmat(f'DNN4SIM_data/DIV2K_recons_snr{snr}_{i}.mat')['recons_data'])
    #ds.append(sio.loadmat(f'DNN4SIM_data/DIV2K_{i}.mat')['data'])
    ds_wf.append(sio.loadmat(f'DNN4SIM_data/DIV2K_wf_snr{snr}_{i}.mat')['wf_data'])

ds_recons = np.concatenate(ds_recons, axis=0)
#ds = np.concatenate(ds, axis=0)
ds_wf = np.concatenate(ds_wf, axis=0)

In [4]:
view = viewer([ds_recons[253], ds_wf[253]], subplots=(1,2), title=['Feature', 'Widefield'], cmap='viridis', joint_zoom=True)

Button(description='Show Widgets', style=ButtonStyle())

In [5]:
for i in range(len(ds_recons)):
    #ds[i] = process_img(apply_rosette(ds[i], rosette_1024))
    ds_recons[i] = process_img(ds_recons[i])
    ds_wf[i] = process_img(ds_wf[i])

In [6]:
view = viewer([ds_recons[62], ds_wf[62]], subplots=(1,2), title=['Feature', 'Widefield'], cmap='viridis', joint_zoom=True)

Button(description='Show Widgets', style=ButtonStyle())

In [9]:
with open(f'DNN4SIM_data/features_snr{snr}.npy', 'wb') as f:
    np.save(f, ds_recons)
#with open(f'DNN4SIM_data/labels_snr{snr}.npy', 'wb') as f:
#    np.save(f, ds)
with open(f'DNN4SIM_data/wf_snr{snr}.npy', 'wb') as f:
    np.save(f, ds_wf)

In [7]:
with open(f'DNN4SIM_data/features_snr{snr}_1.npy', 'wb') as f:
    np.save(f, ds_recons[:ds_recons.shape[0]//2])
with open(f'DNN4SIM_data/features_snr{snr}_2.npy', 'wb') as f:
    np.save(f, ds_recons[ds_recons.shape[0]//2:])
    
#with open(f'DNN4SIM_data/labels_snr{snr}_1.npy', 'wb') as f:
#    np.save(f, ds[:ds.shape[0]//2])
#with open(f'DNN4SIM_data/labels_snr{snr}_2.npy', 'wb') as f:
#    np.save(f, ds[ds.shape[0]//2:])
    
with open(f'DNN4SIM_data/wf_snr{snr}_1.npy', 'wb') as f:
    np.save(f, ds_wf[:ds_wf.shape[0]//2])
with open(f'DNN4SIM_data/wf_snr{snr}_2.npy', 'wb') as f:
    np.save(f, ds_wf[ds_wf.shape[0]//2:])

### Generating mixed dataset

In [19]:
ds_snr = []
snrs = ['15', '20']
ds_type = 'wf'
num = 2
m = len(snrs)
for i in range(m):
    ds_snr.append(np.load(f'DNN4SIM_data/{ds_type}_snr{snrs[i]}_{num}.npy'))

In [20]:
plt.close('all')
view = viewer([ds_snr[0][253], ds_snr[1][253]], subplots=(1,2), joint_zoom=True)

Button(description='Show Widgets', style=ButtonStyle())

In [21]:
ds_mixed = []
for i in range(len(ds_snr[0])):
    ds_mixed.append(ds_snr[m-1-i%m][i])

In [22]:
plt.close('all')
n = 253
view = viewer([ds_mixed[n], ds_mixed[n+1], ds_mixed[n+2], ds_mixed[n+3]], subplots=(2,2), joint_zoom=True)

Button(description='Show Widgets', style=ButtonStyle())

In [23]:
# 1: 5, 10, 20, 35; 2: 5, 10, 15, 20; 3: 15, 20
with open(f'DNN4SIM_data/{ds_type}_snrMIX3_{num}.npy', 'wb') as f:
    np.save(f, ds_mixed)

# Real test image

In [37]:
img_sim = skio.imread('DNN4SIM_data/real_images/cos7_utubule488.tif')
img_recons = cv.resize(skio.imread('DNN4SIM_data/real_images/cos7_utubule488_SIM.tif'), (512,512), interpolation=cv.INTER_CUBIC )
img_wf = skio.imread('DNN4SIM_data/real_images/cos7_utubule488_wfMean.tif')
print(img_sim.shape, img_recons.shape, img_wf.shape)

(9, 512, 512) (512, 512) (512, 512)


In [38]:
img_recons[img_recons<0] = 0
img_wf[img_wf<0] = 0
img_recons = (img_recons-np.min(img_recons))/(np.max(img_recons)-np.min(img_recons))
img_wf = (img_wf-np.min(img_wf))/(np.max(img_wf)-np.min(img_wf))

In [40]:
cv.imwrite('DNN4SIM_data/sim_test_img_real.png', img_recons*255)
cv.imwrite('DNN4SIM_data/sim_test_img_real_wf.png', img_wf*255)

True

In [39]:
plt.close('all')
view = viewer([img_recons, img_wf], joint_zoom=True, cmap='viridis', subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

In [7]:
images = [x for x in img_sim]
view = viewer(images, subplots=(3,3))

Button(description='Show Widgets', style=ButtonStyle())